In [71]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.neighbors import LocalOutlierFactor

In [72]:
data = pd.read_csv('data.csv', delimiter=';')

In [54]:
data['Data'] = pd.to_datetime(data['Data'])
data['Ano'] = data['Data'].dt.year
data['Mes'] = data['Data'].dt.month
data['Dia'] = data['Data'].dt.day
data.drop('Data', axis=1, inplace=True)

In [55]:
enCodigoer = OneHotEncoder(sparse=False)
enCodigoed_data = enCodigoer.fit_transform(data[['Codigo']])
enCodigoed_data_df = pd.DataFrame(enCodigoed_data, columns=enCodigoer.get_feature_names_out(['Codigo']))

data = pd.concat([data.drop(['Codigo'], axis=1), enCodigoed_data_df], axis=1)
#data = pd.concat([data.drop(['Cidade', 'Codigo'], axis=1)], axis=1)

In [56]:
data.drop('Cidade', axis=1, inplace=True)

In [57]:
scaler = RobustScaler()
data[['Pressao Maxima','Pressao Maxima','Temperatura Maxima','Temperatura Minima','Temperatura Orvalho Maxima','Temperatura Orvalho Minima','Umidade Minima',
      'Umidade Maxima','Precipitacao Total', 'Pressao Media', 'Temperatura Media', 'Temperatura Orvalho Media', 'Umidade Media', 'Direcao Vento',
      'Rajada Maxima de Vento','Vento Velocidade Media', 'Latitude', 'Longitude', 'Ano', 'Mes', 'Dia']] = scaler.fit_transform(
          data[['Pressao Maxima','Pressao Maxima','Temperatura Maxima','Temperatura Minima','Temperatura Orvalho Maxima','Temperatura Orvalho Minima',
                'Umidade Minima', 'Umidade Maxima','Precipitacao Total', 'Pressao Media', 'Temperatura Media', 'Temperatura Orvalho Media', 'Umidade Media', 
                'Direcao Vento', 'Rajada Maxima de Vento', 'Vento Velocidade Media', 'Latitude', 'Longitude', 'Ano', 'Mes', 'Dia']])

In [58]:
X = data.drop('Vai Chover Amanha', axis=1)
y = data['Vai Chover Amanha']

Index(['Precipitacao Total', 'Pressao Media', 'Pressao Maxima',
       'Pressao Minima', 'Temperatura Media', 'Temperatura Orvalho Media',
       'Temperatura Maxima', 'Temperatura Minima',
       'Temperatura Orvalho Maxima', 'Temperatura Orvalho Minima',
       'Umidade Maxima', 'Umidade Minima', 'Umidade Media', 'Direcao Vento',
       'Rajada Maxima de Vento', 'Vento Velocidade Media', 'Latitude',
       'Longitude', 'Ano', 'Mes', 'Dia', 'Codigo_A801', 'Codigo_A802',
       'Codigo_A803', 'Codigo_A804', 'Codigo_A805', 'Codigo_A808',
       'Codigo_A809', 'Codigo_A810', 'Codigo_A811', 'Codigo_A812',
       'Codigo_A813', 'Codigo_A826', 'Codigo_A827', 'Codigo_A828',
       'Codigo_A829', 'Codigo_A830', 'Codigo_A831', 'Codigo_A832',
       'Codigo_A833', 'Codigo_A834', 'Codigo_A836', 'Codigo_A837',
       'Codigo_A838', 'Codigo_A839', 'Codigo_A840', 'Codigo_A844',
       'Codigo_A852', 'Codigo_A853', 'Codigo_A854', 'Codigo_A856',
       'Codigo_A878', 'Codigo_A879', 'Codigo_A880', 'Co

In [74]:
lof = LocalOutlierFactor(contamination=0.1)
yhat = lof.fit_predict(X)
mask = yhat != -1
X = X[mask]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X.columns)

In [59]:
train_data = pd.concat([X_train, y_train], axis=1)

majority_class_data = train_data[train_data['Vai Chover Amanha'] == 'Nao']
miNaority_class_data = train_data[train_data['Vai Chover Amanha'] == 'Sim']
upsampled_miNaority_class = resample(miNaority_class_data, replace=True, n_samples=len(majority_class_data))
train_data = pd.concat([majority_class_data, upsampled_miNaority_class], axis=0)

X_train = train_data.drop('Vai Chover Amanha', axis=1)
y_train = train_data['Vai Chover Amanha']

print(train_data['Vai Chover Amanha'].value_counts())

Nao    113909
Sim    113909
Name: Vai Chover Amanha, dtype: int64


In [67]:
def get_mlp(data_x, data_y):
    mlp = MLPClassifier(hidden_layer_sizes=(50,100,100, 50), activation='relu', max_iter=50, random_state=42)
    mlp.fit(data_x, data_y)
    return mlp

In [43]:
def get_dt(data_x, data_y):
    dt = DecisionTreeClassifier(splitter='best', max_depth=None, criterion='gini')
    dt.fit(data_x, data_y)
    return dt

In [44]:
def get_rf(data_x, data_y):
    rf = RandomForestClassifier(class_weight='balanced', random_state=42)
    rf.fit(X_train, y_train)
    return rf

In [61]:
def plot_result_scikit(model, data_x, data_y):
    print("plot_result_scikit")
    y_pred = model.predict(data_x)
    accuracy = accuracy_score(data_y, y_pred),
    #mse = mean_squared_error(y_test, y_pred)
    FP = np.sum((y_pred == 'Sim') & (data_y == 'Nao'))
    FN = np.sum((y_pred == 'Nao') & (data_y == 'Sim'))
    VP = np.sum((y_pred == 'Sim') & (data_y == 'Sim'))
    VN = np.sum((y_pred == 'Nao') & (data_y == 'Nao'))

    print(f"Accuracy: {accuracy} | FP:{FP} | FN:{FN} | VP:{VP} | VN:{VN}")

In [69]:
scikit_model = get_mlp(X_train, y_train)
plot_result_scikit(scikit_model, X_train,y_train) 
plot_result_scikit(scikit_model, X_test,y_test)

c:\Users\Public\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


plot_result_scikit
Accuracy: (0.696187307412057,) | FP:30276 | FN:38938 | VP:74971 | VN:83633
plot_result_scikit
Accuracy: (0.7119630109493686,) | FP:7606 | FN:3732 | VP:7148 | VN:20877
